### 라이브러리

In [56]:
import pandas as pd
import numpy as np
import datetime as dt
import seaborn as sns

import matplotlib.pyplot as plt
import matplotlib 
matplotlib.rcParams['axes.unicode_minus'] = False
matplotlib.rcParams['font.family'] = "AppleGothic"

from sklearn.ensemble import RandomForestRegressor

from pycaret.regression import *
import catboost as cb
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
from sklearn.inspection import permutation_importance
from sklearn.metrics import mean_absolute_error

### 데이터 처리

In [57]:
train = pd.read_csv('G:\내 드라이브\Github\Dacon_Prediction-of-number-AI/train.csv')
test = pd.read_csv('G:\내 드라이브\Github\Dacon_Prediction-of-number-AI/test.csv')
submission = pd.read_csv('G:\내 드라이브\Github\Dacon_Prediction-of-number-AI/sample_submission.csv')

In [58]:
weekdayrankfordinner = {
    '월' : 1,
    '화' : 2,
    '수' : 4,
    '목' : 3,
    '금' : 5
} 

train['요일(석식)'] = train['요일'].map(weekdayrankfordinner)
test['요일(석식)'] = test['요일'].map(weekdayrankfordinner)

In [59]:
train['일자'] = pd.to_datetime(train['일자'])
test['일자'] = pd.to_datetime(test['일자'])

train['년'] = train['일자'].dt.year
train['월'] = train['일자'].dt.month
train['일'] = train['일자'].dt.day
train['주'] = train['일자'].dt.isocalendar().week
train['요일'] = train['일자'].dt.weekday


test['년'] = test['일자'].dt.year
test['월'] = test['일자'].dt.month
test['일'] = test['일자'].dt.day
test['주'] = test['일자'].dt.isocalendar().week
test['요일'] = test['일자'].dt.weekday

train['주'] = train['주'].astype('int') # '주'의 타입이 Uint로 사용이 안되어 int로 변경해줌.
test['주'] = test['주'].astype('int')

In [60]:
drops = ['조식메뉴', '중식메뉴', '석식메뉴']

train = train.drop(drops, axis=1)
test = test.drop(drops, axis=1)

In [61]:
train['식사가능자수'] = (train['본사정원수'] - train['본사휴가자수'] - train['현본사소속재택근무자수'])
test['식사가능자수'] = (test['본사정원수'] - test['본사휴가자수'] - test['현본사소속재택근무자수'])

In [62]:
month1 = {
    1:3,
    2:1,
    3:2,
    4:6,
    5:7,
    6:8,
    7:10,
    8:9,
    9:5,
    10:4,
    11:11,
    12:12
}

month2 = {
    1:11,
    2:2,
    3:1,
    4:4,
    5:8,
    6:6,
    7:9,
    8:7,
    9:5,
    10:3,
    11:10,
    12:12
}
train['월(중식)'] = train['월'].map(month1)
test['월(중식)'] = test['월'].map(month1)

train['월(석식)'] = train['월'].map(month2)
test['월(석식)'] = test['월'].map(month2)

train['월(중식)'] = train['월(중식)'].astype('int') # float형이라 int로 변경
test['월(중식)'] = test['월(중식)'].astype('int')
train['월(석식)'] = train['월(석식)'].astype('int')
test['월(석식)'] = test['월(석식)'].astype('int')

In [63]:
train['연휴'] = 0
test['연휴'] = 0

In [64]:
train['연휴'][4] = 1 
train['연휴'][5] = 2 
train['연휴'][17] = 1 
train['연휴'][18] = 2 
#train['연휴'][47] = 1
#train['연휴'][48] = 2 
#train['연휴'][62] = 1 
#train['연휴'][63] = 2 
#train['연휴'][82] = 1 
#train['연휴'][83] = 2 
train['연휴'][131] = 1 
train['연휴'][132] = 2 
train['연휴'][152] = 1 
train['연휴'][153] = 2 
#train['연휴'][162] = 1 
#train['연휴'][163] = 2 
#train['연휴'][166] = 1 
#train['연휴'][167] = 2 
#train['연휴'][220] = 1 
train['연휴'][221] = 1 
train['연휴'][222] = 2 
train['연휴'][245] = 1
train['연휴'][246] = 2
#train['연휴'][266] = 1
#train['연휴'][267] = 2
#train['연휴'][308] = 1
train['연휴'][309] = 1
train['연휴'][310] = 3
train['연휴'][311] = 2
train['연휴'][330] = 1
train['연휴'][331] = 2
train['연휴'][379] = 3
train['연휴'][380] = 2
#train['연휴'][469] = 1 
train['연휴'][470] = 1 
train['연휴'][471] = 2 
train['연휴'][502] = 3
train['연휴'][503] = 2 
train['연휴'][511] = 3 
train['연휴'][512] = 2
#train['연휴'][555] = 1 
#train['연휴'][556] = 2 
train['연휴'][565] = 3
train['연휴'][566] = 2
#train['연휴'][577] = 1
#train['연휴'][578] = 2
#train['연휴'][651] = 1
#train['연휴'][652] = 2
train['연휴'][705] = 3
#train['연휴'][706] = 2
train['연휴'][708] = 1
train['연휴'][709] = 3
train['연휴'][710] = 2
train['연휴'][732] = 1
train['연휴'][733] = 2
#train['연휴'][748] = 1
#train['연휴'][749] = 2
train['연휴'][815] = 3 #꿀 휴가자리
train['연휴'][816] = 2 #휴가 다음날
train['연휴'][792] = 1 
train['연휴'][793] = 2 
train['연휴'][864] = 3 
train['연휴'][865] = 2 
train['연휴'][882] = 1 
train['연휴'][883] = 2 
train['연휴'][951] = 1 
#train['연휴'][952] = 2 
train['연휴'][953] = 3
train['연휴'][955] = 1
train['연휴'][956] = 2 
train['연휴'][971] = 1 
train['연휴'][972] = 2
#train['연휴'][1027] = 1 
#train['연휴'][1028] = 2 
train['연휴'][1038] = 3
train['연휴'][1039] = 2
train['연휴'][1128] = 1 
train['연휴'][1129] = 3 
train['연휴'][1130] = 2 
train['연휴'][1187] = 3 
train['연휴'][1188] = 2

test['연휴'][10] = 3
test['연휴'][11] = 2
test['연휴'][20] = 1
test['연휴'][21] = 2

In [65]:
# 원핫인코딩
train = pd.get_dummies(train, columns=['연휴'])
test = pd.get_dummies(test, columns=['연휴'])


In [66]:
# test['공휴일전후_0'][20] = 1
# test['공휴일전후_1'][20] = 0

In [67]:
train[train.석식계>0]['석식계'].mean() # 0이아닌값 평균

# 현재 1월만 대체
train['석식계'][244] = 478.8605851979346

#train['석식계'][281] = 398
train['석식계'][492] = 478.8605851979346

#train['석식계'][502] = 398
#train['석식계'][510] = 398
#train['석식계'][529] = 398
train['석식계'][730] = 478.8605851979346

#train['석식계'][747] = 398
#train['석식계'][766] = 398
train['석식계'][973] = 478.8605851979346

#train['석식계'][993] = 398

# 1 2 3 월 자기개발의날 대체.

### 최종제출 ( 자기개발1대체,금)

In [68]:
train.loc[(train.요일==4), '석식계'] -= 10

In [69]:
from catboost import CatBoostRegressor
cat = CatBoostRegressor()

In [70]:
train.columns

Index(['일자', '요일', '본사정원수', '본사휴가자수', '본사출장자수', '본사시간외근무명령서승인건수',
       '현본사소속재택근무자수', '중식계', '석식계', '요일(석식)', '년', '월', '일', '주', '식사가능자수',
       '월(중식)', '월(석식)', '연휴_0', '연휴_1', '연휴_2', '연휴_3'],
      dtype='object')

In [71]:
# train = train[train.석식계!= 0]

x1_train = train[['요일', '본사시간외근무명령서승인건수', '식사가능자수','월(중식)', '일','연휴_0', '연휴_1', '연휴_2', '연휴_3','본사출장자수']] #중식계
x2_train = train[['요일(석식)', '본사시간외근무명령서승인건수', '식사가능자수','월(석식)', '일','연휴_0', '연휴_1', '연휴_2', '연휴_3','본사출장자수']] #석식계

y1_train = train['중식계']
y2_train = train['석식계']

x1_test = test[['요일', '본사시간외근무명령서승인건수', '식사가능자수', '월(중식)', '일','연휴_0', '연휴_1', '연휴_2', '연휴_3','본사출장자수']]
x2_test = test[['요일(석식)', '본사시간외근무명령서승인건수', '식사가능자수', '월(석식)', '일','연휴_0', '연휴_1', '연휴_2', '연휴_3','본사출장자수']]

In [72]:
model1 = cat
model2 = cat

model1.fit(x1_train, y1_train) #중식계
pred1 = model1.predict(x1_test)

model2.fit(x2_train, y2_train) #석식계
pred2 = model2.predict(x2_test)

submission['중식계'] = pred1
submission['석식계'] = pred2

Learning rate set to 0.040347
0:	learn: 203.8591402	total: 3.4ms	remaining: 3.39s
1:	learn: 198.6432905	total: 4.98ms	remaining: 2.49s
2:	learn: 194.2363700	total: 6.53ms	remaining: 2.17s
3:	learn: 189.5655629	total: 8.17ms	remaining: 2.04s
4:	learn: 184.9668869	total: 10.6ms	remaining: 2.12s
5:	learn: 181.6697132	total: 11.5ms	remaining: 1.9s
6:	learn: 177.5285092	total: 13.1ms	remaining: 1.86s
7:	learn: 173.7437382	total: 14.7ms	remaining: 1.82s
8:	learn: 169.9720230	total: 16.3ms	remaining: 1.8s
9:	learn: 166.3996649	total: 17.9ms	remaining: 1.77s
10:	learn: 163.0514662	total: 19.7ms	remaining: 1.77s
11:	learn: 160.3069296	total: 22.4ms	remaining: 1.84s
12:	learn: 157.0998360	total: 24ms	remaining: 1.82s
13:	learn: 154.4056113	total: 25.7ms	remaining: 1.81s
14:	learn: 151.7366654	total: 27.4ms	remaining: 1.8s
15:	learn: 149.2117731	total: 29.1ms	remaining: 1.79s
16:	learn: 147.1192505	total: 30.7ms	remaining: 1.77s
17:	learn: 144.7769380	total: 32.4ms	remaining: 1.77s
18:	learn: 142

In [73]:
submission.head(3)

,일자,중식계,석식계
0,2021-01-27,1009.374780,304.442936
1,2021-01-28,960.607567,494.831344
2,2021-01-29,578.075586,195.156070


In [75]:
submission

,일자,중식계,석식계
0,2021-01-27,1009.374780,304.442936
1,2021-01-28,960.607567,494.831344
2,2021-01-29,578.075586,195.156070
3,2021-02-01,1196.502710,519.061463
4,2021-02-02,1035.536262,556.311066
5,2021-02-03,1048.945482,511.355753
6,2021-02-04,1000.001448,564.061768
7,2021-02-05,737.371194,319.369405
8,2021-02-08,1268.318806,678.139032
9,2021-02-09,1086.712746,575.323272


In [76]:
submission['석식계'].mean()

516.155178884989

In [77]:
submission.to_csv('자기개발1대체,금다운공휴.csv', index=False)